In [2]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.1 MB/s eta 0:00:00


In [4]:
import ast
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
tokenizer = AutoTokenizer.from_pretrained("edwardjross/xlm-roberta-base-finetuned-recipe-all")
model = AutoModelForTokenClassification.from_pretrained("edwardjross/xlm-roberta-base-finetuned-recipe-all")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [6]:
def get_food_indices(sentence):
    # Tokenise and get the token labels
    tokens = tokenizer(sentence, return_tensors="pt").to(device)
    sentence_tokens = tokenizer.tokenize(sentence)
    output = model(**tokens).logits
    predicted_token_class_ids = output.argmax(-1)
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
    # Only assumtption is that "salt pepper is not an ingredient", there will be some space, special character bw 2 different ingredients.
    extractions = []
    ind = 1
    while(ind<len(predicted_tokens_classes)-1):
      if predicted_tokens_classes[ind] == "I-NAME":
        word = sentence_tokens[ind-1]
        # If there is a "▁" at the begining, replace it.
        word = word.replace("▁","")
        # print(word)
        i = ind+1
        # Then loop ahead to see for multiword. 
        while(i<(len(predicted_tokens_classes)-1) and predicted_tokens_classes[i] == "I-NAME"):
          part = sentence_tokens[i-1]
          # If begins with ▁, then needs to be appended to the previous word. 
          if(part[0] != "▁"):
            word= word+part
          # Otherwise, it is another word. 
          else:
            if word == "":
              word = part[1:]
            else:
              word = word+" "+part[1:]
          i+=1
        ind = i
        # Final ingredient formed, append it to the list. 
        extractions.append(word)
      else:
        ind+=1
    return extractions

In [7]:
def get_ingredient(description):
    # 1. Lowercase
    description = description.lower()
    # 2. Remove special characters
    description = description.replace("-","")
    description = description.replace("®","")
    extractions = get_food_indices(description)
    # Lemmatize each word. 
    lemmatizer = WordNetLemmatizer()
    names = []
    for ingredient in extractions:
      ingredient_name = []
      for word in ingredient.split(" "):
        ingredient_name.append(lemmatizer.lemmatize(word))
      ingredient_name = " ".join(ingredient_name)
      names.append(ingredient_name)
    return names

In [8]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.0 MB/s eta 0:00:00


In [9]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [10]:
supercook_ingredients = pd.read_csv('/content/drive/MyDrive/IR Project/ingredients_supercook.csv')
supercook = supercook_ingredients["ingredients"]
supercook = list(set(supercook))

In [13]:
ingDf = pd.read_csv('/content/drive/MyDrive/IR Project/ingredients_dictionary.csv', header = None)

In [14]:
ingDf.head()

,0,1,2
0,dark soy,('dark soy sauce',90)
1,water,('watermelon juice',90)
2,sauce,('pad thai sauce',90)
3,spaghetti,('spaghetti',100)
4,salt,('seasoned salt',90)


In [15]:
# Column names to be added
column_names=["ingredient","match_with","match_percent"]

# Create DataFrame by assigning column names
ingDf.columns = column_names

In [17]:
ingDf.head()

,ingredient,match_with,match_percent
0,dark soy,('dark soy sauce',90)
1,water,('watermelon juice',90)
2,sauce,('pad thai sauce',90)
3,spaghetti,('spaghetti',100)
4,salt,('seasoned salt',90)


In [28]:
ingredient_dict = {}

In [18]:
from tqdm import tqdm

In [29]:
for i, j in tqdm(ingDf.iterrows()):
    ing_name = j['ingredient']
    match_name = j['match_with'][2:-1] 
    if type(j['match_percent']) == float: # nan
        continue
    match_percent = int(j['match_percent'][1:-1])
    if ing_name not in ingredient_dict:
        ingredient_dict[ing_name] = (match_name, match_percent)

30943it [00:02, 12297.21it/s]


In [ ]:
# ingredient_dict['hoisin sauce'] = ('hoisin sauce', 100)

In [30]:
len(ingredient_dict)

30941

In [ ]:
# op = process.extractOne('sale', supercook, scorer = fuzz.WRatio)

In [31]:
import pickle

In [32]:
with open ('/content/drive/MyDrive/IR Project/ingredient_dict.pickle', 'wb') as f:
    pickle.dump(ingredient_dict, f)

In [33]:
ingredient_dict = {}

In [34]:
with open ('/content/drive/MyDrive/IR Project/ingredient_dict.pickle', 'rb') as f:
    ingredient_dict = pickle.load(f)

In [35]:
len(ingredient_dict)

30941

In [36]:
ingredient_dict['tomato']

('tomato', 100)

In [37]:
for ii in [4]: # [1, 2, 3, 4]:
    for jj in [2, 3, 4]: # [1, 2, 3, 4]
        # if ii == 4 and jj < 2:
        #     continue

        print(ii, jj)
        
        filename = '/content/drive/MyDrive/IR Project/_' + str(ii) + '_' + str(jj) + '_AllRecipes.csv'
        recipes = pd.read_csv(filename)
        ingredients = recipes["Ingredients"]
        ingredients = list(ingredients)
        for i in range(len(ingredients)):
            ingredients[i] = ingredients[i][2:-2].split(sep="', '", maxsplit=-1)

        # NER
        ner_ingredients = []
        for i in tqdm(range(len(ingredients))):
            dish_ingredients = ""
            for j in range(len(ingredients[i])):
                ingredient = get_ingredient(ingredients[i][j])
                for k in ingredient:
                    dish_ingredients += k + ', '
            ner_ingredients.append(dish_ingredients)

        ner_ingredients2 = list(i[:-2] for i in ner_ingredients) # remove comma from the end
        recipes['ner_ingredients'] = ner_ingredients2

        # write to csv
        write_filename = '/content/drive/MyDrive/IR Project/_' + str(ii) + '_' + str(jj) + '_AllRecipes_ner.csv' # ner appended to the name
        recipes.to_csv(write_filename)

        # Fuzzy matching
        ner_ingredients = recipes["ner_ingredients"]
        ner_ingredients = list(ner_ingredients)
        
        final_ingredients = []
        for dish_ingredients in tqdm(ner_ingredients):
            if(type(dish_ingredients) != float):
                dish_ingredients = dish_ingredients.split(sep=", ", maxsplit=-1)
                for i in dish_ingredients:
                    if i not in ingredient_dict:
                        # find best match using fuzzy matching
                        op = process.extractOne(i, supercook, scorer = fuzz.WRatio)
                        ingredient_dict[i] = op
        
        # store dictionary as csv
        with open('/content/drive/MyDrive/IR Project/ingredients_dictionary.csv', 'w') as f:
            for key in ingredient_dict.keys():
                f.write("%s,%s\n"%(key, ingredient_dict[key]))

        with open ('/content/drive/MyDrive/IR Project/ingredient_dict.pickle', 'wb') as f:
            pickle.dump(ingredient_dict, f)     

4 2


100%|██████████| 4843/4843 [00:02<00:00, 2199.80it/s]


4 3


100%|██████████| 4842/4842 [03:05<00:00, 26.09it/s]


4 4


100%|██████████| 4845/4845 [03:02<00:00, 26.56it/s]


In [ ]:
len(ingredient_dict)

31431

In [ ]:
for i in ingredient_dict:
    if ingredient_dict[i][1] == 0:
        print(i, ingredient_dict[i])
    # break

3 ('strawberry milk', 0)
nan ('strawberry milk', 0)
2 ('strawberry milk', 0)
2% ('strawberry milk', 0)
60 ('strawberry milk', 0)
9 ('strawberry milk', 0)
 ('honey mustard', 0)
41 ('honey mustard', 0)


In [4]:
with open ('/content/drive/MyDrive/IR Project/ingredient_dict.pickle', 'rb') as f:
    ingredient_dict1 = pickle.load(f)

In [5]:
len(ingredient_dict1)

33581